In [1]:
#Python下数值型与字符型类别变量独热编码（One-hot Encoding）实现
#类别变量的独热编码（One-hot Encoding）
#在数据处理与分析领域，数值型与字符型类别变量的编码是不可或缺的预处理操作。
#本文基于Python下OneHotEncoder与pd.get_dummies两种方法，对机器学习中最优的编码方法——独热编码加以实现。
#1 OneHotEncoder
import pandas as pd
from sklearn.preprocessing import OneHotEncoder #OneHotEncoder是我们实现独热编码的关键模块。

#导入并显示数据前五行
test_data_1=pd.read_csv('datasets/onehot_test.csv',names=['Petrol_tax','Average_income','Paved_Highways'],header=0)
test_data_1.head(5)
#数据前五行展示如下图。其中，前两列'Average_income'与'Paved_Highways'为数值型连续变量，而'Petrol_tax'为数值型类别变量。我们要做的，也就是将第三列'Petrol_tax'进行独热编码。

,Petrol_tax,Average_income,Paved_Highways
0,9.0,3571,1976
1,9.0,4092,1250
2,9.0,3865,1586
3,7.5,4870,2351
4,8.0,4399,431


In [2]:
#接下来，进行独热编码的配置
ohe = OneHotEncoder(handle_unknown='ignore') #是对独热编码的配置
ohe.fit(test_data_1) #对我们刚刚导入的数据进行独热编码处理

#得到一个独热编码配置的输出结果:

OneHotEncoder(handle_unknown='ignore')

In [4]:
#看看独热编码处理后，将我们的数据分成了哪些类别。
ohe.categories_

[array([ 5.  ,  6.  ,  6.58,  7.  ,  7.5 ,  8.  ,  8.5 ,  9.  , 10.  ]),
 array([3063, 3333, 3357, 3448, 3528, 3571, 3601, 3635, 3640, 3656, 3718,
        3721, 3745, 3802, 3846, 3865, 3897, 4045, 4092, 4188, 4206, 4207,
        4258, 4296, 4300, 4318, 4332, 4341, 4345, 4391, 4399, 4447, 4449,
        4476, 4512, 4574, 4593, 4716, 4817, 4870, 4897, 4983, 5002, 5126,
        5215, 5319, 5342], dtype=int64),
 array([  431,   602,  1250,  1333,  1586,  1976,  2138,  2302,  2351,
         2449,  2611,  2619,  3274,  3495,  3635,  3905,  3942,  3985,
         4083,  4121,  4639,  4650,  4686,  4725,  4746,  5399,  5915,
         5939,  5975,  6010,  6385,  6524,  6580,  6594,  6905,  6930,
         7834,  8159,  8507,  8508,  8577,  9061,  9794, 10340, 11868,
        14186, 17782], dtype=int64)]

In [5]:
#可以发现，一共有三个array，为什么呢？仔细看可以发现，独热编码是将我们导入的三列数据全部都当作类别变量来处理了。
#之所以会这样，是因为我们在一开始没有表明哪一列是类别变量，需要进行独热编码；而哪一列不是类别变量，从而不需要进行独热编码。

#那么，我们如何实现上述需求，告诉程序我们要对哪一行进行独热编码呢？
#在老版本的sklearn中，我们可以借助categorical_features=[x]参数来实现这一功能，但是新版本sklearn取消了这一参数。
#那么此时，一方面，我们可以借助ColumnTransformer来实现这一过程，另一方面，我们可以直接对需要进行转换的列加以处理。
#后者相对较为容易理解，因此本文对后者进行讲解。


#我们将test_data_1中的'Petrol_tax'('Type')列作为索引，从而仅仅对该列数据加以独热编码。
ohe_column=pd.DataFrame(ohe.fit_transform(test_data_1[['Petrol_tax']]).toarray())
ohe_column.head(5)

,0,1,2,3,4,5,6,7,8
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [6]:
#可以看到，原来的'Petrol_tax'列现在成为了9列的编码列，那么这样的话，说明我们原先的'Petrol_tax'应该一共是有9个不同的数值。
#是不是这个样子呢？我们来检查一下。
count=pd.DataFrame(test_data_1['Petrol_tax'].value_counts())
print(count)

       Petrol_tax
7.00           19
8.00           10
9.00            8
7.50            4
8.50            3
10.00           1
6.58            1
5.00            1
6.00            1


In [7]:
#好的，没有问题：可以看到此结果共有9行，也就是'Petrol_tax'列原本是有9个不同的值的，证明我们的独热编码没有出错。
#此时看一下我们的test_data_1数据目前长什么样子。
test_data_1.head(5)

,Petrol_tax,Average_income,Paved_Highways
0,9.0,3571,1976
1,9.0,4092,1250
2,9.0,3865,1586
3,7.5,4870,2351
4,8.0,4399,431


In [8]:
#我们仅仅对'Petrol_tax'列做了处理，没有影响到整个初始数据。那么先将原本的Petrol_tax'列剔除掉。
test_data_1=test_data_1.drop(['Petrol_tax'],axis=1)
test_data_1.head(5)

,Average_income,Paved_Highways
0,3571,1976
1,4092,1250
2,3865,1586
3,4870,2351
4,4399,431


In [12]:
#再将经过独热编码处理后的9列加上。
test_data_1.join(ohe_column).head(5)

,Average_income,Paved_Highways,0,1,2,3,4,5,6,7,8
0,3571,1976,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,4092,1250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,3865,1586,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,4870,2351,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,4399,431,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [10]:
#大功告成！
#但是这里还有一个问题，我们经过独热编码所得的列名称始以数字来命名的，非常不方便。
#因此，有没有什么办法可以在独热编码进行的同时，自动对新生成的列加以重命名呢？

In [11]:
#2 pd.get_dummies
#pd.get_dummies是一个最好的办法！其具体用法与上述OneHotEncoder类似，因此具体过程就不再赘述啦，大家看代码就可以明白。
#首先还是导入与上述内容中一致的初始数据。
test_data_2 = pd.read_csv('datasets/onehot_test.csv',names=['Petrol_tax','Average_income','Paved_Highways'],header=0)
test_data_2.head(5)

,Petrol_tax,Average_income,Paved_Highways
0,9.0,3571,1976
1,9.0,4092,1250
2,9.0,3865,1586
3,7.5,4870,2351
4,8.0,4399,431


In [13]:
#进行独热编码并看看结果。
test_data_2_ohe=pd.get_dummies(test_data_2,columns=['Petrol_tax'])
test_data_2_ohe.head(5)

,Average_income,Paved_Highways,Petrol_tax_5.0,Petrol_tax_6.0,Petrol_tax_6.58,Petrol_tax_7.0,Petrol_tax_7.5,Petrol_tax_8.0,Petrol_tax_8.5,Petrol_tax_9.0,Petrol_tax_10.0
0,3571,1976,0,0,0,0,0,0,0,1,0
1,4092,1250,0,0,0,0,0,0,0,1,0
2,3865,1586,0,0,0,0,0,0,0,1,0
3,4870,2351,0,0,0,0,1,0,0,0,0
4,4399,431,0,0,0,0,0,1,0,0,0


In [ ]:
#最终结果中，列名称可以说是非常醒目，同时，
#共有2+9列数据，自动删除了原本的'Petrol_tax'列，实现了“独热编码”“新列重命名”与“原始列删除”，可谓一举三得，简直是太方便啦~